In [ ]:
import sys
if 'google.colab' in sys.modules:
    print("Mounting Google Drive")
    from google.colab import drive
    drive.mount('/content/drive')
    print("Installing, please wait.")
    !pip install --quiet accelerate diffusers torch torchvision
    # Until v4.26.0 is out, requires:
    !pip install --upgrade git+https://github.com/huggingface/transformers
    # Sometimes, I may have to use bleeding edge:
    # !pip install -U git+https://github.com/huggingface/diffusers
    !git clone --quiet https://github.com/maruel/ml
    sys.path.insert(0, "/content/ml/notebooks/stable_diffusion")
import torch
print(torch.cuda.get_device_name(0), "/ %.2fGB" % (torch.cuda.get_device_properties(0).total_memory/1024/1024/1024))

In [ ]:
import time

import IPython
import numpy as np

import sdcommon
import depth2img

def run(save=True, **kwargs):
    ml = depth2img.ML()
    params = depth2img.Params(**kwargs)
    start = time.time()
    imgs, depth = ml.run(params, local_files_only=False)
    print("Took %.1fs" % (time.time()-start))
    if save:
      print("Saved as %s.png" % sdcommon.save(ml, params, depth, *imgs)) 
    return imgs, depth

def loop(**kwargs):
    out = []
    iterations = sdcommon.unroll(**kwargs)
    print("Doing %d iterations" % len(iterations))
    for label, line in iterations:
        imgs, _ = run(**line)
        img = imgs[0]
        out.append((label, img))
        print(label)
        IPython.display.display(img)
    return out

out = loop(
    imagename="out/txt2img00000.png",
    prompt="Christmas",
    neg_prompt="Disfigured. Too many fingers.",
    strength=[0.4, 0.5],
    seed=range(7, 9),
    steps=25)

In [ ]:
import ipyplot
ipyplot.plot_images([i[1] for i in out], [i[0] for i in out], img_width=200)